# 问答模型微调

##  Step1 导包

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import random 
import math
from torch.utils.data import Dataset,DataLoader
import pandas as pd
from datasets import load_dataset , Dataset , load_from_disk
from transformers import AutoTokenizer
from datasets import concatenate_datasets
import os


## Step2 导入数据

In [2]:
webQA_train_files = [r'G:\Anaconda\Kaggle\gpt2\data\webQA\train.json' , r'G:\Anaconda\Kaggle\gpt2\data\webQA\dev.json']
du_train_files = [r'G:\Anaconda\Kaggle\gpt2\data\dureader\train.csv' , r'G:\Anaconda\Kaggle\gpt2\data\dureader\dev.csv']

In [3]:
webQA_train = load_dataset('json', data_files=webQA_train_files , split='train')
du_train = load_dataset('csv' ,data_files= du_train_files , split = 'train')
ds = Dataset.load_from_disk(r"\Anaconda\Kaggle\LLm\transformers-code-master\02-NLP Tasks\16-generative_chatbot\alpaca_data_zh")

In [4]:
webQA_train[0]

{'input': '世界上最早的报纸诞生于',
 'output': '中国。北宋末年(公元11,12世纪)出现的印刷报纸,不仅是中国新闻史上最早的印刷报纸,也是世界新闻史上最早的印刷报纸.中国新闻事业历史的悠久,内容的丰富,是任何西方国家都难以比肩的.<e>中国古代的报纸产生于中国的封建社会时期,是封建地主阶级及其政治代表占统治地位的封建自然经济通过新闻手段的反映.在漫长的封建社会时期,中国古代的报纸,不论是官方的邸报,还是民办的小报和京报,都必然要和当时的封建统治者保持一定的联系,受他们的制约.官方的邸报固然是封建统治阶级的喉舌和御用的宣传工具,民办的小报和京报也只能在封建统治阶级的控制下活动,不能越雷池一步.封建统治者绝不允许可以自由报道一切消息和自由发表一切意见的报纸存在.中国古代的报纸在为当时的读者提供朝野政治和社会信息方面确实起过一定的作用,但始终没有摆脱统治阶级的掌握.中国古代报纸的历史,基本上是一部封建统治阶级掌握传播媒介,控制舆论工具,限制言论出版自由的历史.<e>中国古代的邸报有1200年左右的历史.小报有近千年的历史.民间报房出版的邸报,京报有近400年的历史.它们从诞生到结束,持续的时间都不算短,但发展不快,形式内容的变化不大.',
 'id': 0}

In [5]:

du_train[0]

{'text1': '第35集[SEP]第35集雪见缓缓张开眼睛，景天又惊又喜之际，长卿和紫萱的仙船驶至，见众人无恙，也十分高兴。众人登船，用尽合力把自身的真气和水分输给她。雪见终于醒过来了，但却一脸木然，全无反应。众人向常胤求助，却发现人世界竟没有雪见的身世纪录。长卿询问清微的身世，清微语带双关说一切上了天界便有答案。长卿驾驶仙船，众人决定立马动身，往天界而去。众人来到一荒山，长卿指出，魔界和天界相连。由魔界进入通过神魔之井，便可登天。众人至魔界入口，仿若一黑色的蝙蝠洞，但始终无法进入。后来花楹发现只要有翅膀便能飞入。于是景天等人打下许多乌鸦，模仿重楼的翅膀，制作数对翅膀状巨物。刚佩戴在身，便被吸入洞口。众人摔落在地，抬头发现魔界守卫。景天和众魔套交情，自称和魔尊重楼相熟，众魔不理，打了起来。',
 'text2': '仙剑奇侠传3第几集上天界'}

In [6]:
def process_func4ds(example):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(r'G:\Anaconda\Kaggle\gpt2\tokenizer\bytebpe_zh')
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["提问: " + example["instruction"], example["input"]]).strip() + "\n\n回答: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
def process_func4web(example):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(r'G:\Anaconda\Kaggle\gpt2\tokenizer\bytebpe_zh')
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join("提问: " + example["input"]).strip() + "\n\n回答: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
def process_func4du(example):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(r'G:\Anaconda\Kaggle\gpt2\tokenizer\bytebpe_zh')
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join("提问: " + example["text2"]).strip() + "\n\n回答: ")
    response = tokenizer(example["text1"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [11]:
tokenized_ds = ds.map(process_func4ds ,remove_columns=ds.column_names,num_proc = 6)

Map (num_proc=6):   0%|          | 0/26858 [00:00<?, ? examples/s]

In [14]:
tokenized_web = webQA_train.map(process_func4web ,remove_columns=webQA_train.column_names,num_proc = 6)

Map (num_proc=6):   0%|          | 0/39192 [00:00<?, ? examples/s]

In [10]:
tokenized_du = du_train.map(process_func4du , remove_columns=du_train.column_names , num_proc=6)

Map (num_proc=6):   0%|          | 0/15937 [00:00<?, ? examples/s]

In [16]:
merged = concatenate_datasets([tokenized_ds, tokenized_web, tokenized_du])
merged = merged.shuffle(seed=42)

out_dir = r"G:\Anaconda\Kaggle\gpt2\data\tokenized_merged"
os.makedirs(out_dir, exist_ok=True)
merged.save_to_disk(out_dir)

Saving the dataset (0/1 shards):   0%|          | 0/81987 [00:00<?, ? examples/s]

In [17]:
def _len_batch(batch):
    return {"len": [len(x) for x in batch["input_ids"]]}

lens = merged.map(_len_batch, batched=True, batch_size=1000, num_proc=6, remove_columns=[])
total_tokens = int(sum(lens["len"]))
print("tokens_total:", total_tokens)

Map (num_proc=6):   0%|          | 0/81987 [00:00<?, ? examples/s]

tokens_total: 10035888


In [18]:
if "labels" in merged.column_names:
    def _valid_batch(batch):
        return {"valid": [sum(1 for t in x if t != -100) for x in batch["labels"]]}
    vlen = merged.map(_valid_batch, batched=True, batch_size=1000, num_proc=6, remove_columns=[])
    total_valid = int(sum(vlen["valid"]))
    print("tokens_valid:", total_valid)

Map (num_proc=6):   0%|          | 0/81987 [00:00<?, ? examples/s]

tokens_valid: 7369978


In [2]:
merged = load_from_disk(r'G:\Anaconda\Kaggle\gpt2\data\tokenized_merged')

## Step3 训练模型

In [3]:
from model import GPTModel
class LMDataset(Dataset):
    def __init__(self, data_dir):
        import os
        import glob
        import datasets
        self._pylist = None
        try:
            ds = datasets.load_from_disk(data_dir)
            if isinstance(ds, datasets.DatasetDict):
                ds = ds.get("train", list(ds.values())[0])
            self.ds = ds
        except Exception:
            pattern = os.path.join(data_dir, "data-*.arrow")
            files = sorted(glob.glob(pattern))
            if not files:
                raise
            try:
                ds = datasets.Dataset.from_file(files[0])
                self.ds = ds
            except Exception:
                import pyarrow.ipc as pa_ipc
                try:
                    reader = pa_ipc.open_file(files[0])
                    table = reader.read_all()
                except Exception:
                    reader = pa_ipc.open_stream(files[0])
                    table = reader.read_all()
                col = table.column("input_ids")
                self._pylist = col.to_pylist()
                self.ds = None
    def __len__(self):
        return len(self._pylist) if self._pylist is not None else len(self.ds)
    def __getitem__(self, idx):
        if self._pylist is not None:
            return {"input_ids": self._pylist[idx]}
        ex = self.ds[idx]
        return {"input_ids": ex["input_ids"]}

def load_tokenizer(tokenizer_dir):
    import os
    from transformers import AutoTokenizer, GPT2TokenizerFast
    try:
        tok = AutoTokenizer.from_pretrained(tokenizer_dir)
    except Exception:
        vocab = os.path.join(tokenizer_dir, "vocab.json")
        merges = os.path.join(tokenizer_dir, "merges.txt")
        tok = GPT2TokenizerFast(vocab_file=vocab, merges_file=merges)
    if tok.pad_token_id is None:
        try:
            tok.add_special_tokens({"pad_token": "<|pad|>"})
        except Exception:
            pass
    return tok

class LMDataCollator:
    def __init__(self, context_length, pad_id):
        self.context_length = context_length
        self.pad_id = pad_id
    def __call__(self, batch):
        ids = []
        for item in batch:
            x = torch.tensor(item["input_ids"], dtype=torch.long)
            if x.shape[0] >= self.context_length:
                x = x[: self.context_length]
            else:
                pad = torch.full((self.context_length - x.shape[0],), self.pad_id, dtype=torch.long)
                x = torch.cat([x, pad], dim=0)
            ids.append(x)
        input_ids = torch.stack(ids, dim=0)
        attention_mask = (input_ids != self.pad_id).long()
        labels = input_ids.roll(-1, dims=1)
        labels[:, -1] = -100
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

def build_lm_dataloader(data_dir, tokenizer_dir, batch_size, context_length, shuffle=True, num_workers=0):
    tok = load_tokenizer(tokenizer_dir)
    pad_id = tok.pad_token_id if tok.pad_token_id is not None else 0
    ds = LMDataset(data_dir)
    collate = LMDataCollator(context_length, pad_id)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate)

def get_vocab_size(tokenizer_dir):
    tok = load_tokenizer(tokenizer_dir)
    try:
        return int(len(tok))
    except Exception:
        return int(tok.vocab_size)

def compute_lm_loss(logits, labels):
    return F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), ignore_index=-100)

In [4]:
import os, numpy as np, torch
from torch.utils.data import Dataset, DataLoader

class PackedBinDataset(Dataset):
    def __init__(self, prefix):
        # .idx 是二进制索引：int64 offset_tokens, int32 length_tokens
        self.bin_path = prefix + ".bin"
        self.idx_path = prefix + ".idx"
        self.entries = []
        with open(self.idx_path, "rb") as f:
            while True:
                rec = f.read(12)
                if not rec:
                    break
                off = int.from_bytes(rec[:8], "little", signed=True)
                ln  = int.from_bytes(rec[8:], "little", signed=True)
                self.entries.append((off, ln))
        # 用 memmap 读 .bin（int32）
        self.mm = np.memmap(self.bin_path, dtype=np.int32, mode="r")

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        off, ln = self.entries[idx]
        arr = self.mm[off: off + ln]          # shape: (ctx_len,)
        input_ids = torch.from_numpy(arr.astype(np.int64))   # to int64 tensor
        return {"input_ids": input_ids}

def collate_packed(batch):
    # 固定 ctx_len，无 padding。attention_mask 全 1；labels 右移一位，最后一位 -100
    input_ids = torch.stack([b["input_ids"] for b in batch], dim=0)
    attention_mask = torch.ones_like(input_ids, dtype=torch.long)
    labels = input_ids.roll(-1, dims=1)
    labels[:, -1] = -100
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [5]:
import os, numpy as np, torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils import clip_grad_norm_
from torch.cuda.amp import GradScaler, autocast
from tqdm.auto import tqdm
from model import GPTModel, compute_lm_loss, get_vocab_size, load_tokenizer

import os, numpy as np, torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils import clip_grad_norm_
from torch.amp import GradScaler, autocast
from torch.optim.lr_scheduler import LambdaLR
import numpy as np
import math
from tqdm.auto import tqdm
from model import GPTModel, compute_lm_loss, get_vocab_size, load_tokenizer

class PackedBinDataset(Dataset):
    def __init__(self, prefix):
        self.bin_path = prefix + ".bin"
        self.idx_path = prefix + ".idx"
        self.entries = []
        with open(self.idx_path, "rb") as f:
            while True:
                rec = f.read(12)
                if not rec: break
                off = int.from_bytes(rec[:8], "little", signed=True)
                ln  = int.from_bytes(rec[8:], "little", signed=True)
                self.entries.append((off, ln))
        self.mm = np.memmap(self.bin_path, dtype=np.int32, mode="r")
    def __len__(self): return len(self.entries)
    def __getitem__(self, idx):
        off, ln = self.entries[idx]
        arr = self.mm[off: off + ln]
        return {"input_ids": torch.from_numpy(arr.astype(np.int64))}

def collate_packed(batch):
    input_ids = torch.stack([b["input_ids"] for b in batch], dim=0)
    attention_mask = torch.ones_like(input_ids, dtype=torch.long)
    labels = input_ids.roll(-1, dims=1)
    labels[:, -1] = -100
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

import os, math, torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils import clip_grad_norm_
from tqdm.auto import tqdm
from model import build_lm_dataloader, GPTModel, compute_lm_loss, get_vocab_size

def notebook_train_hfds(
    data_dir,
    tokenizer_dir,
    ctx_len=1024,
    emb_dim=768, n_heads=12, n_layers=12, dropout=0.1, qkv_bias=False, tie_weights=True,
    batch_size=8, num_workers=0,
    lr=3e-4, weight_decay=0.1, warmup_steps=3000, max_steps=0, epochs=1, grad_accum=1,
    log_every=50, save_every=1000, save_dir=r"G:\Anaconda\Kaggle\gpt2\checkpoints", resume=""
):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dl = build_lm_dataloader(data_dir, tokenizer_dir, batch_size, ctx_len, shuffle=True, num_workers=num_workers)
    b = next(iter(dl))
    max_id = int(b["input_ids"].max().item())
    vocab_nominal = get_vocab_size(tokenizer_dir)
    vocab_eff = max(vocab_nominal, max_id + 1)
    cfg = {
        "vocab_size": vocab_eff,
        "context_length": ctx_len,
        "emb_dim": emb_dim,
        "dropout": dropout,
        "n_heads": n_heads,
        "qkv_bias": qkv_bias,
        "n_layers": n_layers,
    }
    model = GPTModel(cfg).to(device)
    if tie_weights:
        model.output_layer.weight = model.embedding_layer.embedding.weight
    decay, no_decay = [], []
    for n, p in model.named_parameters():
        if not p.requires_grad:
            continue
        if n.endswith("bias") or ("ln" in n.lower()) or ("layernorm" in n.lower()):
            no_decay.append(p)
        else:
            decay.append(p)
    optim = AdamW([
        {"params": decay, "weight_decay": weight_decay},
        {"params": no_decay, "weight_decay": 0.0},
    ], lr=lr)
    total_steps = max_steps if (max_steps and max_steps > 0) else (epochs * len(dl))
    def lr_lambda(step):
        if step < warmup_steps:
            return step / max(1, warmup_steps)
        return max(0.0, (total_steps - step) / max(1, total_steps - warmup_steps))
    scheduler = LambdaLR(optim, lr_lambda)
    try:
        scaler = torch.amp.GradScaler('cuda' if device == 'cuda' else 'cpu')
    except Exception:
        from torch.cuda.amp import GradScaler as _GS
        scaler = _GS()
    start_step = 0
    if resume:
        ckpt = torch.load(resume, map_location="cpu")
        model.load_state_dict(ckpt.get("model", {}), strict=False)
        try:
            optim.load_state_dict(ckpt.get("optimizer", {}))
        except Exception:
            pass
        try:
            scaler.load_state_dict(ckpt.get("scaler", {}))
        except Exception:
            pass
        start_step = int(ckpt.get("step", 0))
    os.makedirs(save_dir, exist_ok=True)
    model.train()
    step = start_step
    running, count = 0.0, 0
    p_epoch = tqdm(total=total_steps, desc="训练", unit="step")
    for _ in range(epochs):
        for batch in dl:
            step += 1
            if device == "cuda":
                try:
                    from torch.amp import autocast as _ac
                    _cuda_ctx = _ac('cuda')
                except Exception:
                    from torch.cuda.amp import autocast as _ac
                    _cuda_ctx = _ac()
                with _cuda_ctx:
                    logits = model(batch["input_ids"].to(device), attention_mask=batch["attention_mask"].to(device))
                    loss = compute_lm_loss(logits, batch["labels"].to(device)) / grad_accum
            else:
                logits = model(batch["input_ids"], attention_mask=batch["attention_mask"])
                loss = compute_lm_loss(logits, batch["labels"]) / grad_accum
            scaler.scale(loss).backward()
            if step % grad_accum == 0:
                scaler.unscale_(optim)
                clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optim)
                scaler.update()
                optim.zero_grad(set_to_none=True)
                scheduler.step()
            running += loss.item()
            count += 1
            if log_every and step % log_every == 0:
                avg = running * grad_accum / max(1, count)
                ppl = math.exp(avg) if avg < 700 else float('inf')
                lr_now = scheduler.get_last_lr()[0]
                valid_tokens = int((batch["labels"] != -100).sum().item())
                p_epoch.set_postfix_str(f"loss {avg:.4f} | ppl {ppl:.2f} | lr {lr_now:.6f} | valid {valid_tokens}")
                running, count = 0.0, 0
            if save_every and step % save_every == 0:
                path = os.path.join(save_dir, f"ckpt_step_{step}.pt")
                _obj = {
                    "model": model.state_dict(),
                    "optimizer": optim.state_dict(),
                    "scaler": scaler.state_dict(),
                    "step": step,
                    "cfg": cfg,
                }
                try:
                    torch.save(_obj, path)
                except RuntimeError:
                    torch.save(_obj, path, _use_new_zipfile_serialization=False)
            p_epoch.update(1)
            if step >= total_steps:
                break
    p_epoch.close()
    return model

def save_final_model(model, save_dir=r'G:\Anaconda\Kaggle\gpt2\checkpoints', file_name="final_model.pt"):
    os.makedirs(save_dir, exist_ok=True)
    final_ckpt = os.path.join(save_dir, file_name)
    sd_cpu = {}
    for k, v in tqdm(model.state_dict().items(), desc="准备最终权重", unit="param"):
        sd_cpu[k] = v.detach().cpu()
    try:
        torch.save({'model': sd_cpu}, final_ckpt)
    except RuntimeError:
        torch.save({'model': sd_cpu}, final_ckpt, _use_new_zipfile_serialization=False)
    return final_ckpt


In [6]:
model = notebook_train_hfds(
  data_dir=r"G:\Anaconda\Kaggle\gpt2\data\tokenized_merged",
  tokenizer_dir=r"G:\Anaconda\Kaggle\gpt2\tokenizer\bytebpe_zh",
  ctx_len=1024,
  emb_dim=768, n_heads=12, n_layers=12, dropout=0.1, qkv_bias=False, tie_weights=True,
  batch_size=1, num_workers=0,
  lr=3e-4, weight_decay=0.1, warmup_steps=3000, max_steps=0, epochs=1, grad_accum=1,
  log_every=50, save_every=1000,
  save_dir=r"G:\Anaconda\Kaggle\gpt2\checkpoints",
  resume=r"G:\Anaconda\Kaggle\gpt2\checkpoints\final_model.pt"
)

final_path = save_final_model(model, save_dir=r"G:\Anaconda\Kaggle\gpt2\checkpoints", file_name="final_model_step3.pt")
print(final_path)

训练:   0%|          | 0/81987 [00:00<?, ?step/s]

KeyboardInterrupt: 

In [22]:
from train_hfds import qa_generate

answer = qa_generate(
  question="提问：中国科技应该如何发展？ 回答：",
  ckpt_path=r"G:\Anaconda\Kaggle\gpt2\checkpoints\ckpt_step_2000.pt",   # 或 final_model.pt
  tokenizer_dir=r"G:\Anaconda\Kaggle\gpt2\tokenizer\bytebpe_zh",
  max_new_tokens=512,
  temperature=0.8,
  top_k=30,
  top_p=0.8,
  device='cuda',
  use_half=True,
  ctx_len=1024, emb_dim=768, n_heads=12, n_layers=12, dropout=0.1, qkv_bias=False, tie_weights=True
)
print(answer)

美国科技应该提供一个关于网络安全信息，并根据用户信息进行进行分析，以便用户信息进行分析。
